In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
from utils import load_file, run_genetic_algorithm, plot_log, decode_chromosome, plot_graph, chromosome_fitness
import seaborn as sns
sns.set()

In [ ]:
E, D, W, d, g, v, t, a, c = load_file('data/test_10.txt')

In [ ]:
from utils import run_genetic_algorithm
%time population, solutions, log = run_genetic_algorithm(chromosome_fitness, \
                                E, D, W, d, g, v, t, a, c, \
                                pop_size=100, n_generations=20, \
                                n_solutions=5, crossover_p=0.5, \
                                mutation_p=0.2)

In [ ]:
%time !glpsol --math data/test_10_ampl.mod

## Dashboard widgets

In [3]:
from IPython.display import display
import ipywidgets as widgets

In [4]:
inp_pop_size = widgets.IntSlider(100, 50, 300, description='Population size',
                      style={'description_width': 'initial'})
inp_n_generations = widgets.IntSlider(20, 5, 100, description='# Generations',
                      style={'description_width': 'initial'})

In [5]:
import os
sel_dataset_options = {file[:-4]: [file, file.replace('.txt', '_ampl.mod')] \
               for file in sorted(os.listdir('data')) \
               if file.startswith('test_') and \
                  file.endswith('.txt')}
sel_dataset = widgets.Dropdown(
    options = sel_dataset_options,
    description='Dataset',
    value=sel_dataset_options[sorted(list(sel_dataset_options))[0]]
)

In [6]:
btn_run_ga   = widgets.Button(description='Run genetic algorithm')
btn_run_glpk = widgets.Button(description='Run GLPK solver')
output_log    = widgets.Output()
output_plots  = widgets.Output()

In [7]:
optimization_log = None

In [8]:
def run_ga(e):
    global optimization_log
    with output_log:
        E, D, W, d, g, v, t, a, c = load_file(f'data/{sel_dataset.value[0]}')
        population, solutions, log = run_genetic_algorithm(
                                E, D, W, d, g, v, t, a, c, \
                                pop_size=inp_pop_size.value, 
                                n_generations=inp_n_generations.value, \
                                n_solutions=5, crossover_p=0.5, \
                                mutation_p=0.2)  
        optimization_log = log
    output_log.clear_output(wait=True)
btn_run_ga.on_click(run_ga)

In [9]:
btn_plot_opt_log  = widgets.Button(description='Plot optimization log')
btn_plot_solution = widgets.Button(description='Plot solution')

def plot_opt_log(e):
    with output_plots:
        if optimization_log:            
            ax = plot_log(optimization_log, optimal_value=None, min_max=False)
            display(ax.figure)
    output_plots.clear_output(wait=True)
            
btn_plot_opt_log.on_click(plot_opt_log)

In [10]:
def render_dashboard():
    options = widgets.VBox([inp_pop_size,
                            inp_n_generations,
                            sel_dataset])
    actions = widgets.VBox([btn_run_ga,
                            btn_run_glpk])
    plots = widgets.VBox([btn_plot_opt_log,
                          btn_plot_solution])
    input_panel  = widgets.HBox([options, actions, plots])
    output_panel = widgets.HBox([output_log, output_plots])
    return widgets.VBox([input_panel,
                  output_panel])

## Dashboard

In [11]:
render_dashboard()

VBox(children=(HBox(children=(VBox(children=(IntSlider(value=100, description='Population size', max=300, min=50, style=SliderStyle(description_width='initial')), IntSlider(value=20, description='# Generations', min=5, style=SliderStyle(description_width='initial')), Dropdown(description='Dataset', options={'test_10': ['test_10.txt', 'test_10_ampl.mod'], 'test_15': ['test_15.txt', 'test_15_ampl.mod'], 'test_20': ['test_20.txt', 'test_20_ampl.mod'], 'test_30': ['test_30.txt', 'test_30_ampl.mod'], 'test_40': ['test_40.txt', 'test_40_ampl.mod'], 'test_50': ['test_50.txt', 'test_50_ampl.mod']}, value=['test_10.txt', 'test_10_ampl.mod']))), VBox(children=(Button(description='Run genetic algorithm', style=ButtonStyle()), Button(description='Run GLPK solver', style=ButtonStyle()))), VBox(children=(Button(description='Plot optimization log', style=ButtonStyle()), Button(description='Plot solution', style=ButtonStyle()))))), HBox(children=(Output(), Output()))))